In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, isnull, when, array_contains
from pyspark.sql.types import StructType, StructField, StringType

In [8]:
spark = SparkSession.builder \
    .appName("ifood-jsons") \
    .getOrCreate()

In [18]:
file_location = "/Users/alyssonamaral/Documents/ifood-case/case_data/transactions.json"
file_type = "json"

infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

df_transactions = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df_transactions = df_transactions \
    .withColumn("amount", col("value.amount")) \
    .withColumn("offer_id_", col("value.`offer id`")) \
    .withColumn("offer_id", col("value.offer_id")) \
    .withColumn("reward", col("value.reward")) \
    .drop("value")

df_transactions.show(5)

+--------------------+--------------+---------------------+------+--------------------+--------+------+
|          account_id|         event|time_since_test_start|amount|           offer_id_|offer_id|reward|
+--------------------+--------------+---------------------+------+--------------------+--------+------+
|78afa995795e4d85b...|offer received|                  0.0|  NULL|9b98b8c7a33c4b65b...|    NULL|  NULL|
|a03223e636434f42a...|offer received|                  0.0|  NULL|0b1e1539f2cc45b7b...|    NULL|  NULL|
|e2127556f4f64592b...|offer received|                  0.0|  NULL|2906b810c7d441179...|    NULL|  NULL|
|8ec6ce2a7e7949b1b...|offer received|                  0.0|  NULL|fafdcd668e3743c1b...|    NULL|  NULL|
|68617ca6246f4fbc8...|offer received|                  0.0|  NULL|4d5c57ea9a6940dd8...|    NULL|  NULL|
+--------------------+--------------+---------------------+------+--------------------+--------+------+
only showing top 5 rows

